In [219]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

pd.set_option("display.precision", 1)

# set a random seed
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)


## On récupère la donnée

In [220]:
test_customers = pd.read_csv('data/test_customers.csv')
test_locations = pd.read_csv('data/test_locations.csv')

train_customers = pd.read_csv('data/train_customers.csv')
train_locations = pd.read_csv('data/train_locations.csv')

orders = pd.read_csv('data/orders.csv')
vendors = pd.read_csv('data/vendors.csv')


/tmp/ipykernel_9393/3811830009.py:7: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv('data/orders.csv')


In [221]:
print('train_customers shape: ', train_customers.shape)
print('test_customers shape: ', test_customers.shape)
train_customers.head()

train_customers shape:  (34674, 8)
test_customers shape:  (9768, 8)


,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at
0,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23
1,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41
2,S2ALZFL,Male,NaN,0,1,EN,2018-03-14 18:31:43,2018-03-14 18:31:42
3,952DBJQ,Male,NaN,1,1,EN,2018-03-15 19:47:07,2018-03-15 19:47:07
4,1IX6FXS,Male,NaN,1,1,EN,2018-03-15 19:57:01,2018-03-15 19:57:01


In [222]:
print('train_locations shape: ', train_locations.shape)
print('test_locations shape: ', test_locations.shape)
train_locations.head()

train_locations shape:  (59503, 5)
test_locations shape:  (16720, 5)


,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.7,-78.8
1,02SFNJH,1,NaN,1.7,0.8
2,02SFNJH,2,NaN,-0.5,0.7
3,RU43CXC,0,Home,0.1,0.4
4,BDFBPRD,0,NaN,2.5,0.7


In [223]:
print('orders shape: ', orders.shape)
orders.head()

orders shape:  (135303, 26)


,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163238.0,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,105,2019-08-01 05:30:16,0,NaN,92PEE24 X 0 X 105
1,163240.0,QS68UD8,1.0,8.7,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,294,2019-08-01 05:31:10,0,Work,QS68UD8 X 0 X 294
2,163241.0,MB7VY5F,2.0,14.4,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:31:33,0,NaN,MB7VY5F X 0 X 83
3,163244.0,KDJ951Y,1.0,7.1,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,90,2019-08-01 05:34:54,0,Home,KDJ951Y X 0 X 90
4,163245.0,BAL0RVT,4.0,27.2,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:35:51,0,Work,BAL0RVT X 0 X 83


In [224]:
print('vendors shape: ', vendors.shape)
vendors.head()

vendors shape:  (100, 59)


,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,...,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,4,118597.0,-0.6,0.8,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,...,1.0,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1
1,13,118608.0,-0.5,0.7,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,...,1.0,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1
2,20,118616.0,-0.4,0.6,Restaurants,2.0,0.0,8.0,1.0,08:00AM-10:45PM,...,1.0,"4,8,91,10","Breakfast,Desserts,Free Delivery,Indian",Y,1.0,1.0,2018-05-04 22:28:22,2020-04-07 16:35:55,3,1
3,23,118619.0,-0.6,0.8,Restaurants,2.0,0.0,5.0,1.0,10:59AM-10:30PM,...,1.0,"5,8,30,24","Burgers,Desserts,Fries,Salads",Y,1.0,1.0,2018-05-06 19:20:48,2020-04-02 00:56:17,3,1
4,28,118624.0,0.5,0.6,Restaurants,2.0,0.7,15.0,1.0,11:00AM-11:45PM,...,1.0,5,Burgers,Y,1.0,1.0,2018-05-17 22:12:38,2020-04-05 15:57:41,3,1


### On arrange et on nettoie la donnée pour notre réseau

In [225]:
# merge train_customers(akeed_customer_id) and train_locations(customer_id)
train_customers_locations = pd.merge(train_customers, train_locations, left_on='akeed_customer_id', right_on='customer_id', how='inner')
print('train_customers_locations shape: ', train_customers_locations.shape)
train_customers_locations.head()

train_customers_locations shape:  (58213, 13)


,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at,customer_id,location_number,location_type,latitude,longitude
0,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23,TCHWPBT,0,Work,-96.4,-67.2
1,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23,TCHWPBT,1,Home,-96.6,-67.2
2,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23,TCHWPBT,2,NaN,-0.1,-78.6
3,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41,ZGFSYCZ,0,Home,-0.2,-78.6
4,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41,ZGFSYCZ,1,Home,0.2,-78.6


In [226]:
# merge orders(vendor_id) and vendors(id)
orders_vendors = pd.merge(orders, vendors, left_on='vendor_id', right_on='id', how='inner')
print('train_customers_locations_orders_vendors shape: ', orders_vendors.shape)
orders_vendors.head()

train_customers_locations_orders_vendors shape:  (135303, 85)


,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at_y,updated_at,device_type,display_orders
0,163238.0,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,...,1.0,"1,5,91,29,27","American,Burgers,Free Delivery,Hot Dogs,Pasta",Y,1.0,1.0,2018-10-19 12:38:55,2020-04-07 23:41:41,3,1
1,163247.0,I9DNSMJ,2.0,14.4,1,NaN,0.0,NaN,NaN,No,...,1.0,"1,5,91,29,27","American,Burgers,Free Delivery,Hot Dogs,Pasta",Y,1.0,1.0,2018-10-19 12:38:55,2020-04-07 23:41:41,3,1
2,163248.0,I9DNSMJ,2.0,14.4,1,NaN,0.0,NaN,NaN,No,...,1.0,"1,5,91,29,27","American,Burgers,Free Delivery,Hot Dogs,Pasta",Y,1.0,1.0,2018-10-19 12:38:55,2020-04-07 23:41:41,3,1
3,163579.0,4BQI76G,1.0,8.7,1,NaN,0.0,NaN,NaN,No,...,1.0,"1,5,91,29,27","American,Burgers,Free Delivery,Hot Dogs,Pasta",Y,1.0,1.0,2018-10-19 12:38:55,2020-04-07 23:41:41,3,1
4,163609.0,V6JC4VF,4.0,20.4,1,NaN,0.0,NaN,NaN,No,...,1.0,"1,5,91,29,27","American,Burgers,Free Delivery,Hot Dogs,Pasta",Y,1.0,1.0,2018-10-19 12:38:55,2020-04-07 23:41:41,3,1


In [227]:
# same for test data
test_customers_locations = pd.merge(test_customers, test_locations, left_on='akeed_customer_id', right_on='customer_id', how='inner')
print('test_customers_locations shape: ', test_customers_locations.shape)
test_customers_locations.head()

test_customers_locations shape:  (16333, 13)


,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at,customer_id,location_number,location_type,latitude,longitude
0,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,0,NaN,-9.6e+01,-67.2
1,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,1,NaN,3.9e-02,-78.6
2,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,2,Other,-9.5e+01,43.7
3,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,3,Other,-9.6e+01,43.6
4,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,4,Work,2.0e+02,44.1


On créé ici un dataframe ressmeblant a SampleSubmission pour entrainer notre réseau

In [228]:
# keep only the columns we need :CID, LOC_NUM, VENDOR
check_orders = orders_vendors[['customer_id', 'vendor_id', 'LOCATION_NUMBER', 'CID X LOC_NUM X VENDOR']]
check_orders = check_orders[['customer_id','LOCATION_NUMBER', 'vendor_id', 'CID X LOC_NUM X VENDOR']]
check_orders = check_orders.rename(columns={'customer_id': 'CID','LOCATION_NUMBER': 'LOC_NUM', 'vendor_id': 'VENDOR', 'CID X LOC_NUM X VENDOR': 'CID X LOC_NUM X VENDOR'})
check_orders = check_orders.drop_duplicates()
print('check_orders shape: ', check_orders.shape)
check_orders.head(10)

check_orders shape:  (80142, 4)


,CID,LOC_NUM,VENDOR,CID X LOC_NUM X VENDOR
0,92PEE24,0,105,92PEE24 X 0 X 105
1,I9DNSMJ,0,105,I9DNSMJ X 0 X 105
3,4BQI76G,0,105,4BQI76G X 0 X 105
4,V6JC4VF,0,105,V6JC4VF X 0 X 105
5,IJHNJ90,0,105,IJHNJ90 X 0 X 105
6,IB555AJ,0,105,IB555AJ X 0 X 105
7,NVDVNOO,0,105,NVDVNOO X 0 X 105
8,VLXLNZM,0,105,VLXLNZM X 0 X 105
9,OD336VP,0,105,OD336VP X 0 X 105
10,TVSPG2H,0,105,TVSPG2H X 0 X 105


on ajoute une column avec notre prediction, ici la prediction est 1 car on a créé ce df a partir des orders qui ont eu lieux

In [229]:
# avec the number of rows in check_orders
print('number of rows in check_orders: ', check_orders.shape[0])
rows_predict_1 = check_orders.shape[0]

# add a column prediction
check_orders['PREDICTION'] = 1
check_orders.head()

number of rows in check_orders:  80142


,CID,LOC_NUM,VENDOR,CID X LOC_NUM X VENDOR,PREDICTION
0,92PEE24,0,105,92PEE24 X 0 X 105,1
1,I9DNSMJ,0,105,I9DNSMJ X 0 X 105,1
3,4BQI76G,0,105,4BQI76G X 0 X 105,1
4,V6JC4VF,0,105,V6JC4VF X 0 X 105,1
5,IJHNJ90,0,105,IJHNJ90 X 0 X 105,1


on a "rows_predict_1" orders predit, pour entrainer notre reseau on voudrait aussi un nombre de orders du meme ordre de grandeur avec une prediction de 0

In [230]:
# add the same number of rows with PREDICTION = 0
# get the number of rows with PREDICTION = 1
num_rows = check_orders[check_orders['PREDICTION'] == 1].shape[0]

# we need to generate valid pairs of (CID, LOC_NUM, VENDOR) that are not in check_orders
# we can do this by generating all possible pairs and then removing the ones that are in check_orders
# generate all possible pairs, with a maximum of 2 * num_rows
all_pairs = pd.DataFrame(columns=['customer_id', 'location_number', 'vendor_id'])
all_pairs['customer_id'] = np.random.choice(train_customers_locations['customer_id'], 2 * num_rows)
all_pairs['location_number'] = np.random.choice(train_customers_locations['location_number'], 2 * num_rows)
all_pairs['vendor_id'] = np.random.choice(vendors['id'], 2 * num_rows)
all_pairs = all_pairs.drop_duplicates()

print('all_pairs shape: ', all_pairs.shape)

# remove the ones that are in check_orders
all_pairs = all_pairs.merge(check_orders, left_on=['customer_id', 'location_number', 'vendor_id'], right_on=['CID', 'LOC_NUM', 'VENDOR'], how='left')
all_pairs = all_pairs[all_pairs['PREDICTION'].isnull()]

# make sure that the number of rows is the same as in check_orders
all_pairs = all_pairs.drop_duplicates()
print('number of rows in all_pairs: ', all_pairs.shape[0])

# keep rows_predict_1 rows
all_pairs = all_pairs.iloc[:rows_predict_1]

# keep only the columns we need
all_pairs = all_pairs[['customer_id', 'location_number', 'vendor_id']]
all_pairs = all_pairs.rename(columns={'customer_id': 'CID','location_number': 'LOC_NUM', 'vendor_id': 'VENDOR'})
all_pairs['CID X LOC_NUM X VENDOR'] = all_pairs['CID'] + ' X ' + all_pairs['LOC_NUM'].astype(str) + ' X ' + all_pairs['VENDOR'].astype(str)
all_pairs['PREDICTION'] = 0
print('all_pairs shape: ', all_pairs.shape)
all_pairs.head()

all_pairs shape:  (158046, 3)
number of rows in all_pairs:  156191
all_pairs shape:  (80142, 5)


,CID,LOC_NUM,VENDOR,CID X LOC_NUM X VENDOR,PREDICTION
0,4NHBOQ3,1,295,4NHBOQ3 X 1 X 295,0
1,YJK14JS,3,582,YJK14JS X 3 X 582,0
2,MZ60652,0,84,MZ60652 X 0 X 84,0
3,YG315AA,0,161,YG315AA X 0 X 161,0
4,J3AC3A0,0,274,J3AC3A0 X 0 X 274,0


In [231]:
# merge check_orders and all_pairs
all_pairs = pd.concat([check_orders, all_pairs])
print('all_pairs shape: ', all_pairs.shape)
all_pairs.head()

all_pairs shape:  (160284, 5)


,CID,LOC_NUM,VENDOR,CID X LOC_NUM X VENDOR,PREDICTION
0,92PEE24,0,105,92PEE24 X 0 X 105,1
1,I9DNSMJ,0,105,I9DNSMJ X 0 X 105,1
3,4BQI76G,0,105,4BQI76G X 0 X 105,1
4,V6JC4VF,0,105,V6JC4VF X 0 X 105,1
5,IJHNJ90,0,105,IJHNJ90 X 0 X 105,1


avec les infos de CID, LOC_NUM et VENDOR on peut crééer un vecteur d'entrée pour entrainer le reseau sur nos prédictions

In [232]:
# make the df input for the model
# we use CID to merge with train_customers
# we use VENDOR to merge with vendors
# the df must have the same number of rows as all_pairs in the same order
df_input = all_pairs[['CID', 'VENDOR', 'LOC_NUM']]
df_input = df_input.merge(train_customers, left_on='CID', right_on='akeed_customer_id', how='left')
df_input.drop_duplicates(inplace=True, subset=['CID', 'LOC_NUM', 'VENDOR'])
print('df_input shape: ', df_input.shape)
df_input = df_input.merge(vendors, left_on='VENDOR', right_on='id', how='left')
# keep only the columns we need
#df_input = df_input[['CID', 'VENDOR', 'gender', 'dob', '']]

# print the columns names
print('df_input columns: ', df_input.columns)

print('df_input shape: ', df_input.shape)
display(df_input.isnull().sum())
df_input.head()

df_input shape:  (160284, 11)
df_input columns:  Index(['CID', 'VENDOR', 'LOC_NUM', 'akeed_customer_id', 'gender', 'dob',
       'status_x', 'verified_x', 'language_x', 'created_at_x', 'updated_at_x',
       'id', 'authentication_id', 'latitude', 'longitude',
       'vendor_category_en', 'vendor_category_id', 'delivery_charge',
       'serving_distance', 'is_open', 'OpeningTime', 'OpeningTime2',
       'prepration_time', 'commission', 'is_akeed_delivering',
       'discount_percentage', 'status_y', 'verified_y', 'rank', 'language_y',
       'vendor_rating', 'sunday_from_time1', 'sunday_to_time1',
       'sunday_from_time2', 'sunday_to_time2', 'monday_from_time1',
       'monday_to_time1', 'monday_from_time2', 'monday_to_time2',
       'tuesday_from_time1', 'tuesday_to_time1', 'tuesday_from_time2',
       'tuesday_to_time2', 'wednesday_from_time1', 'wednesday_to_time1',
       'wednesday_from_time2', 'wednesday_to_time2', 'thursday_from_time1',
       'thursday_to_time1', 'thursday_from

CID                      0
VENDOR                   0
LOC_NUM                  0
akeed_customer_id     1888
gender               42387
                     ...  
city_id                  0
created_at_y             0
updated_at_y             0
device_type              0
display_orders           0
Length: 70, dtype: int64

,CID,VENDOR,LOC_NUM,akeed_customer_id,gender,dob,status_x,verified_x,language_x,created_at_x,...,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at_y,updated_at_y,device_type,display_orders
0,92PEE24,105,0,92PEE24,Male,NaN,1.0,1.0,EN,2018-11-06 16:46:08,...,1.0,"1,5,91,29,27","American,Burgers,Free Delivery,Hot Dogs,Pasta",Y,1.0,1.0,2018-10-19 12:38:55,2020-04-07 23:41:41,3,1
1,I9DNSMJ,105,0,I9DNSMJ,Male,NaN,1.0,1.0,EN,2018-11-16 02:21:14,...,1.0,"1,5,91,29,27","American,Burgers,Free Delivery,Hot Dogs,Pasta",Y,1.0,1.0,2018-10-19 12:38:55,2020-04-07 23:41:41,3,1
2,4BQI76G,105,0,4BQI76G,NaN,NaN,1.0,1.0,EN,2019-08-08 00:27:34,...,1.0,"1,5,91,29,27","American,Burgers,Free Delivery,Hot Dogs,Pasta",Y,1.0,1.0,2018-10-19 12:38:55,2020-04-07 23:41:41,3,1
3,V6JC4VF,105,0,V6JC4VF,Female,1997.0,1.0,1.0,EN,2018-12-27 20:30:26,...,1.0,"1,5,91,29,27","American,Burgers,Free Delivery,Hot Dogs,Pasta",Y,1.0,1.0,2018-10-19 12:38:55,2020-04-07 23:41:41,3,1
4,IJHNJ90,105,0,IJHNJ90,Male,NaN,1.0,1.0,EN,2018-10-30 00:37:58,...,1.0,"1,5,91,29,27","American,Burgers,Free Delivery,Hot Dogs,Pasta",Y,1.0,1.0,2018-10-19 12:38:55,2020-04-07 23:41:41,3,1


### Nettoyage de la donnée

In [233]:
# keep only the columns we need
df_input = df_input[['gender', 'dob',
       'status_x', 'verified_x', 'language_x',
        'latitude', 'longitude',
       'vendor_category_en', 'vendor_category_id', 'delivery_charge',
       'serving_distance', 'is_open',
       'prepration_time', 'commission', 'is_akeed_delivering',
       'discount_percentage', 'status_y', 'verified_y', 'rank', 'language_y',
       'vendor_rating', 'primary_tags',
       'open_close_flags', 'one_click_vendor',
       'country_id', 'city_id', 'device_type',
       'display_orders',
       # 'vendor_tag', 'vendor_tag_name',
       ]]

# print all the columns with their number of missing values
print('number of missing values per column:')
display(df_input.isnull().sum())

# replace the missing values in gender with "None"
df_input['gender'] = df_input['gender'].fillna('None')

# replace the missing values in dob with the mean
df_input['dob'] = df_input['dob'].fillna(df_input['dob'].mean())

# replace the missing values in language_x with "None"
df_input['language_x'] = df_input['language_x'].fillna('None')

# replace the missing values in commission with the mean
df_input['commission'] = df_input['commission'].fillna(df_input['commission'].mean())

# replace the missing values in Language_y with "None"
df_input['language_y'] = df_input['language_y'].fillna('None')

# replace primary_tags with "None"
df_input['primary_tags'] = df_input['primary_tags'].fillna('None')

# replace vendor_tag_name with "None"
#df_input['vendor_tag_name'] = df_input['vendor_tag_name'].fillna('None')

# replace vendor_tag with "None"
#df_input['vendor_tag'] = df_input['vendor_tag'].fillna('None')


number of missing values per column:


gender                  42387
dob                    145992
status_x                 1888
verified_x               1888
language_x              37611
latitude                    0
longitude                   0
vendor_category_en          0
vendor_category_id          0
delivery_charge             0
serving_distance            0
is_open                     0
prepration_time             0
commission              21911
is_akeed_delivering         0
discount_percentage         0
status_y                    0
verified_y                  0
rank                        0
language_y              21911
vendor_rating               0
primary_tags            36766
open_close_flags            0
one_click_vendor            0
country_id                  0
city_id                     0
device_type                 0
display_orders              0
dtype: int64

In [234]:
# check that the number of rows is the same as in all_pairs
print('number of rows in df_input: ', df_input.shape[0])
print('number of rows in all_pairs: ', all_pairs.shape[0])

number of rows in df_input:  160284
number of rows in all_pairs:  160284


In [235]:
# print the types of the columns
print('df_input types: ', df_input.dtypes)

# label encoding
from sklearn.preprocessing import LabelEncoder

# create a LabelEncoder object
le = LabelEncoder()

df_input['status_x'] = le.fit_transform(df_input['status_x'])
df_input['verified_x'] = le.fit_transform(df_input['verified_x'])
df_input['language_x'] = le.fit_transform(df_input['language_x'])
df_input['gender'] = le.fit_transform(df_input['gender'])
df_input['vendor_category_en'] = le.fit_transform(df_input['vendor_category_en'])
df_input['is_akeed_delivering'] = le.fit_transform(df_input['is_akeed_delivering'])
df_input['language_y'] = le.fit_transform(df_input['language_y'])
df_input['primary_tags'] = le.fit_transform(df_input['primary_tags'])
df_input['one_click_vendor'] = le.fit_transform(df_input['one_click_vendor'])

df_input types:  gender                  object
dob                    float64
status_x               float64
verified_x             float64
language_x              object
latitude               float64
longitude              float64
vendor_category_en      object
vendor_category_id     float64
delivery_charge        float64
serving_distance       float64
is_open                float64
prepration_time          int64
commission             float64
is_akeed_delivering     object
discount_percentage    float64
status_y               float64
verified_y               int64
rank                     int64
language_y              object
vendor_rating          float64
primary_tags            object
open_close_flags       float64
one_click_vendor        object
country_id             float64
city_id                float64
device_type              int64
display_orders           int64
dtype: object


### Création des data splits

In [236]:
# data split
train, test = train_test_split(df_input, test_size=0.2, random_state=42, shuffle=True)
print('test shape: ', test.shape)
print('train shape: ', train.shape)

y_train = all_pairs['PREDICTION']
y_train, y_test = train_test_split(y_train, test_size=0.2, random_state=42, shuffle=True)

test shape:  (32057, 28)
train shape:  (128227, 28)


### Creation du modèle

In [237]:
# creation of an ai model
# it takes as input a dataframe with the columns as in df_input
# it returns a dataframe with the columns as in all_pairs and a column PREDICTION with the prediction

from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization

model = keras.Sequential()
model.add(Input(shape=(df_input.shape[1],)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

model.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 256)               7424      
                                                                 
 dense_39 (Dense)            (None, 128)               32896     
                                                                 
 dense_40 (Dense)            (None, 64)                8256      
                                                                 
 dense_41 (Dense)            (None, 1)                 65        
                                                                 
Total params: 48,641
Trainable params: 48,641
Non-trainable params: 0
_________________________________________________________________


In [238]:
# fit the model
model.fit(train, y_train, epochs=10, batch_size=32, validation_data=(test, y_test))


Epoch 1/10
4008/4008 [==============================] - 4s 811us/step - loss: 0.5005 - accuracy: 0.4995 - val_loss: 0.4957 - val_accuracy: 0.5043
Epoch 2/10
4008/4008 [==============================] - 3s 792us/step - loss: 0.5011 - accuracy: 0.4989 - val_loss: 0.4957 - val_accuracy: 0.5043
Epoch 3/10
1367/4008 [=========>....................] - ETA: 1s - loss: 0.4989 - accuracy: 0.5011

KeyboardInterrupt: 